<a href="https://colab.research.google.com/github/vblagoje/notebooks/blob/main/haystack2x-demos/haystack_rag_serperdev_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction

This notebook showcases the upcoming Haystack OpenAPI service-based Retriever-Augmented Generation (RAG). Given a user query, we search the web for the results of the query and inject these results into LLM context along with the system prompt.

## 1. Setup

Let's install necessary libraries and import key modules to build the foundation for the subsequent steps.

In [1]:
!pip uninstall -y llmx

Found existing installation: llmx 0.0.15a0
Uninstalling llmx-0.0.15a0:
  Successfully uninstalled llmx-0.0.15a0


In [2]:
!pip install -q openapi3 jsonref haystack-ai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.1 MB/s eta 0:00:00


In [11]:
import getpass
import os
import json
import requests

from haystack import Pipeline
from haystack.components.generators.utils import print_streaming_chunk
from haystack.components.converters import OpenAPIServiceToFunctions
from haystack.components.generators.chat import OpenAIChatGenerator
from haystack.components.connectors import OpenAPIServiceConnector
from haystack.dataclasses import ChatMessage
from haystack.utils import Secret

## 2. Prepare SerperDev service along with system prompt

- Generate OpenAI functions definitions for SerperDev

In [4]:
gen_func_pipeline = Pipeline()
gen_func_pipeline.add_component("spec_to_functions", OpenAPIServiceToFunctions())

In [5]:
functions_result = gen_func_pipeline.run(data={"sources":["https://bit.ly/3NIJqnd"],
                                               "system_messages":[requests.get("https://bit.ly/3TdHsyB").text]})

## 3. API keys, set up simple authentication mechanism

In [6]:
llm_api_key = getpass.getpass("Enter LLM provider api key:")


Enter LLM provider api key:··········


In [7]:
serper_dev_key = getpass.getpass("Enter serperdev api key:")
services_auth = {"SerperDev":serper_dev_key}

Enter serperdev api key:··········


## 4. Retrieval step - SerperDev service invocation

In [13]:
invoke_service_pipe = Pipeline()
invoke_service_pipe.add_component("functions_llm", OpenAIChatGenerator(api_key=Secret.from_token(llm_api_key), model="gpt-3.5-turbo-0613"))
invoke_service_pipe.add_component("openapi_container", OpenAPIServiceConnector(services_auth))
invoke_service_pipe.connect("functions_llm.replies", "openapi_container.messages")

In [14]:
user_prompt = "Why was Sam Altman ousted from OpenAI?"

In [15]:
service_desc_document = functions_result["spec_to_functions"]["documents"][0]
openai_functions_definition = json.loads(service_desc_document.content)
openapi_spec = service_desc_document.meta["spec"]

service_response = invoke_service_pipe.run(data={"messages":[ChatMessage.from_user(user_prompt)],
                                                 "generation_kwargs": {"functions": [openai_functions_definition]},
                                                 "service_openapi_spec": openapi_spec})

## 6. Generate LLM response

Inject service response into LLM context, pair it with system prompt

In [19]:
gen_pipe = Pipeline()
llm = OpenAIChatGenerator(api_key=Secret.from_token(llm_api_key), model="gpt-4-1106-preview", streaming_callback=print_streaming_chunk)
gen_pipe.add_component("llm", llm)

github_pr_prompt_messages = [ChatMessage.from_system(service_desc_document.meta["system_message"])] + service_response["openapi_container"]["service_response"]
final_result = gen_pipe.run(data={"messages": github_pr_prompt_messages})

The reason for Sam Altman's ouster from OpenAI was due to concerns about his honesty in communications. The board removed him as CEO after a review found that he had not been "consistently candid in his communications." This was reported by The Washington Post, and the board's decision was said to be triggered by a pattern of manipulation, specifically Altman's attempts to avoid certain issues.